# Applying logic of geocat-comp climatology to a UXarray datafile

And encountering issues with the cftime accessor

geocat-comp code for reference - https://github.com/NCAR/geocat-comp/blob/main/geocat/comp/climatologies.py

In [1]:
import cartopy.crs as ccrs
import holoviews as hv
import uxarray as ux
import xarray as xr
import numpy as np

In [ ]:
grid_path = 'data/ne30pg2_grd.nc'
data_path = 'data/E3SM_ENSO_ctl_keeling_eam_h0_0006-12.nc'

uxds = ux.open_dataset(grid_path, data_path)
uxds

<xarray.UxDataset> Size: 2GB
Dimensions:   (time: 72, lev: 72, n_face: 21600)
Coordinates:
  * time      (time) object 576B 0001-02-01 00:00:00 ... 0007-01-01 00:00:00
  * lev       (lev) float64 576B 0.1238 0.1828 0.2699 ... 986.2 993.8 998.5
Dimensions without coordinates: n_face
Data variables: (12/26)
    TREFHT    (time, n_face) float32 6MB ...
    PSL       (time, n_face) float32 6MB ...
    TS        (time, n_face) float32 6MB ...
    PRECC     (time, n_face) float32 6MB ...
    PRECL     (time, n_face) float32 6MB ...
    LHFLX     (time, n_face) float32 6MB ...
    ...        ...
    Q         (time, lev, n_face) float32 448MB ...
    U         (time, lev, n_face) float32 448MB ...
    V         (time, lev, n_face) float32 448MB ...
    Z3        (time, lev, n_face) float32 448MB ...
    FLUT      (time, n_face) float32 6MB ...
    FLUTC     (time, n_face) float32 6MB ...

We already know that for this dataset the time coordinate is called 'time'.

Looking at surface temperature data 'TS'

In [3]:
uxds["TS"].isel(time=1).plot()

:Image   [x,y]   (x_y TS)

One piece of geocat-comp's climatology is inferring a time coordinate if it is not provided. While not necessary to move forward with this dataset exploration, it will be required for making the function work more broadly.

It fails with `AttributeError: 'UxDataset' object has no attribute 'cf'`.

Time exists and is of type `cftime.DatetimeNoLeap`, can't convert to pandas datatimeindex as in https://docs.xarray.dev/en/stable/generated/xarray.CFTimeIndex.to_datetimeindex.html

In [4]:
seasons_dict = {
    "DJF": ([12, 1, 2], 'QS-DEC'),
    "JFM": ([1, 2, 3], 'QS-JAN'),
    "FMA": ([2, 3, 4], 'QS-FEB'),
    "MAM": ([3, 4, 5], 'QS-MAR'),
    "AMJ": ([4, 5, 6], 'QS-APR'),
    "MJJ": ([5, 6, 7], 'QS-MAY'),
    "JJA": ([6, 7, 8], 'QS-JUN'),
    "JAS": ([7, 8, 9], 'QS-JUL'),
    "ASO": ([8, 9, 10], 'QS-AUG'),
    "SON": ([9, 10, 11], 'QS-SEP'),
    "OND": ([10, 11, 12], 'QS-OCT'),
    "NDJ": ([11, 12, 1], 'QS-NOV'),
}
seasons = ['DJF', 'JJA', 'MAM', 'SON']
time_dim = 'time'
frequency = 'season'

Fails at filtering data to desired season

In [5]:
seasonal_climates = []
for season in seasons:

    # Grab the months for each season
    months, quarter_season = seasons_dict[season]

    # Filter data to only contain the months of interest
    dset_filter = uxds.sel(
        {time_dim: uxds[time_dim].dt.month.isin(months)})

    # Calculate monthly average before calculating seasonal climatologies
    dset_filter = dset_filter.resample({time_dim: quarter_season}).mean().dropna(time_dim)

    # Compute the weights for the months in each season so that the
    # seasonal averages account for months being of different lengths
    month_length = dset_filter[time_dim].dt.days_in_month
    weights = month_length / month_length.sum()
    climatology = (dset_filter * weights).sum(dim=time_dim)

    seasonal_climates.append(climatology)

In [6]:
seasonal_climates

[<xarray.UxDataset> Size: 66MB
 Dimensions:   (lev: 72, n_face: 21600)
 Coordinates:
   * lev       (lev) float64 576B 0.1238 0.1828 0.2699 ... 986.2 993.8 998.5
 Dimensions without coordinates: n_face
 Data variables: (12/26)
     TREFHT    (n_face) float64 173kB 292.6 292.0 293.5 ... 281.0 279.5 280.6
     PSL       (n_face) float64 173kB 1.015e+05 1.016e+05 ... 1.021e+05 1.022e+05
     TS        (n_face) float64 173kB 293.4 292.8 294.2 ... 288.6 283.5 286.4
     PRECC     (n_face) float64 173kB 1.668e-08 1.896e-08 ... 3.937e-09 1.941e-09
     PRECL     (n_face) float64 173kB 1.499e-08 1.762e-08 ... 5.636e-08 4.509e-08
     LHFLX     (n_face) float64 173kB 82.58 79.22 83.3 79.5 ... 259.2 134.8 164.0
     ...        ...
     Q         (lev, n_face) float64 12MB 1.344e-06 1.344e-06 ... 0.004275
     U         (lev, n_face) float64 12MB -6.629 -4.365 -9.505 ... 3.74 3.591
     V         (lev, n_face) float64 12MB 12.0 12.33 11.15 ... -1.143 -1.264
     Z3        (lev, n_face) float64 12

In [7]:
uxds_seasonal = ux.concat(seasonal_climates, dim='season')
uxds_seasonal.coords['season'] = np.array(seasons).astype(object)

AttributeError: The first object does not have a 'uxgrid' attribute.

Previous error due xr.concat. Will reach out more, have to re-associate the ux.grid with resulting uxgrid. use ux.concat

https://uxarray.readthedocs.io/en/latest/generated/uxarray.concat.html#uxarray.concat

I need to check higher up stream to where uxarray grid info is lost.

In [ ]:
uxds_seasonal

AttributeError: 'NoneType' object has no attribute '_ds'

<xarray.UxDataset> Size: 263MB
Dimensions:   (season: 4, n_face: 21600, lev: 72)
Coordinates:
  * lev       (lev) float64 576B 0.1238 0.1828 0.2699 ... 986.2 993.8 998.5
  * season    (season) object 32B 'DJF' 'JJA' 'MAM' 'SON'
Dimensions without coordinates: n_face
Data variables: (12/26)
    TREFHT    (season, n_face) float64 691kB 292.6 292.0 293.5 ... 294.8 295.6
    PSL       (season, n_face) float64 691kB 1.015e+05 1.016e+05 ... 1.013e+05
    TS        (season, n_face) float64 691kB 293.4 292.8 294.2 ... 295.9 297.0
    PRECC     (season, n_face) float64 691kB 1.668e-08 1.896e-08 ... 2.361e-08
    PRECL     (season, n_face) float64 691kB 1.499e-08 1.762e-08 ... 3.514e-08
    LHFLX     (season, n_face) float64 691kB 82.58 79.22 83.3 ... 87.39 93.37
    ...        ...
    Q         (season, lev, n_face) float64 50MB 1.344e-06 1.344e-06 ... 0.01365
    U         (season, lev, n_face) float64 50MB -6.629 -4.365 ... -0.4419
    V         (season, lev, n_face) float64 50MB 12.0 12.33 .

Why is it giving attribute error and then printing out exactly what I'd expect?

In [8]:
seasonal_climates[0]

AttributeError: 'NoneType' object has no attribute '_ds'

<xarray.UxDataset> Size: 66MB
Dimensions:   (lev: 72, n_face: 21600)
Coordinates:
  * lev       (lev) float64 576B 0.1238 0.1828 0.2699 ... 986.2 993.8 998.5
Dimensions without coordinates: n_face
Data variables: (12/26)
    TREFHT    (n_face) float64 173kB 292.6 292.0 293.5 ... 281.0 279.5 280.6
    PSL       (n_face) float64 173kB 1.015e+05 1.016e+05 ... 1.021e+05 1.022e+05
    TS        (n_face) float64 173kB 293.4 292.8 294.2 ... 288.6 283.5 286.4
    PRECC     (n_face) float64 173kB 1.668e-08 1.896e-08 ... 3.937e-09 1.941e-09
    PRECL     (n_face) float64 173kB 1.499e-08 1.762e-08 ... 5.636e-08 4.509e-08
    LHFLX     (n_face) float64 173kB 82.58 79.22 83.3 79.5 ... 259.2 134.8 164.0
    ...        ...
    Q         (lev, n_face) float64 12MB 1.344e-06 1.344e-06 ... 0.004275
    U         (lev, n_face) float64 12MB -6.629 -4.365 -9.505 ... 3.74 3.591
    V         (lev, n_face) float64 12MB 12.0 12.33 11.15 ... -1.143 -1.264
    Z3        (lev, n_face) float64 12MB 6.436e+04 6.43

Unpack the for loop again

In [9]:
months, quarter_season = seasons_dict['DJF']

    # Filter data to only contain the months of interest
dset_filter = uxds.sel(
        {time_dim: uxds[time_dim].dt.month.isin(months)})

    # Calculate monthly average before calculating seasonal climatologies
dset_filter = dset_filter.resample({time_dim: quarter_season}).mean().dropna(time_dim)

    # Compute the weights for the months in each season so that the
    # seasonal averages account for months being of different lengths
month_length = dset_filter[time_dim].dt.days_in_month
weights = month_length / month_length.sum()
climatology = (dset_filter * weights).sum(dim=time_dim)

Error seems to be right away?

In [10]:
dset_filter

AttributeError: 'NoneType' object has no attribute '_ds'

<xarray.UxDataset> Size: 230MB
Dimensions:   (lev: 72, time: 7, n_face: 21600)
Coordinates:
  * lev       (lev) float64 576B 0.1238 0.1828 0.2699 ... 986.2 993.8 998.5
  * time      (time) object 56B 0000-12-01 00:00:00 ... 0006-12-01 00:00:00
Dimensions without coordinates: n_face
Data variables: (12/26)
    TREFHT    (time, n_face) float32 605kB 294.1 293.5 295.0 ... 281.4 282.5
    PSL       (time, n_face) float32 605kB 1.014e+05 1.014e+05 ... 1.026e+05
    TS        (time, n_face) float32 605kB 294.9 294.3 295.6 ... 285.3 288.0
    PRECC     (time, n_face) float32 605kB 1.536e-08 2.328e-08 ... 1.331e-09
    PRECL     (time, n_face) float32 605kB 1.133e-08 1.173e-08 ... 5.002e-08
    LHFLX     (time, n_face) float32 605kB 81.37 82.7 71.43 ... 131.5 154.9
    ...        ...
    Q         (time, lev, n_face) float32 44MB 1.384e-06 1.384e-06 ... 0.004927
    U         (time, lev, n_face) float32 44MB -9.551 -7.213 ... 2.664 2.119
    V         (time, lev, n_face) float32 44MB 13.46 13.